# Homenet

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homenet").config("spark.memory.offHeap.enabled", "true").config(
    "spark.memory.offHeap.size", "10g").getOrCreate()

In [19]:
adresser = spark.read.json("spark-warehouse/adresser.json", multiLine=True)
adresser.createOrReplaceTempView("adresser")

maklare = spark.read.json("spark-warehouse/maklare.json", multiLine=True)
maklare.createOrReplaceTempView("maklare")

objekt = spark.read.json("spark-warehouse/objekt.json", multiLine=True)
objekt.createOrReplaceTempView("objekt")

personer = spark.read.json("spark-warehouse/personer.json", multiLine=True)
personer.createOrReplaceTempView("personer")

In [20]:
objekt.printSchema()

root
 |-- adress: long (nullable = true)
 |-- antal_rum: long (nullable = true)
 |-- avslutat: string (nullable = true)
 |-- balkong: string (nullable = true)
 |-- forsaljningspris: long (nullable = true)
 |-- hiss: string (nullable = true)
 |-- id: long (nullable = true)
 |-- kopare: long (nullable = true)
 |-- maklararvode: long (nullable = true)
 |-- maklare: long (nullable = true)
 |-- manadsavgift: long (nullable = true)
 |-- saljare: long (nullable = true)
 |-- saljstart: string (nullable = true)
 |-- storlek: long (nullable = true)
 |-- utgangspris: long (nullable = true)
 |-- vaning: long (nullable = true)


In [21]:
objekt.show(1,0)

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|adress|antal_rum|avslutat  |balkong|forsaljningspris|hiss|id |kopare|maklararvode|maklare|manadsavgift|saljare|saljstart |storlek|utgangspris|vaning|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+
|75    |6        |2020-07-19|1      |14561000        |1   |1  |5     |218415      |2      |3090        |107    |2020-01-25|137    |12420000   |5     |
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+


In [22]:
result = spark.sql("SELECT * FROM personer LIMIT 5")
result.show()

+------+---------+--------------------+-------+---+----------+----------+
|adress|efternamn|               email|fornamn| id|  personnr| telefonnr|
+------+---------+--------------------+-------+---+----------+----------+
|   102| Wudeland|mwudeland0@drupal...|Danièle|  1|1997-12-19|3114705221|
|   104|   Prayer|jprayer1@wordpres...|    Zoé|  2|1958-02-26|6162426013|
|   106|  Dimmick|   hdimmick2@epa.gov|Thérèse|  3|1988-07-25|5095529405|
|   108| Cuddehay|scuddehay3@delici...| Noémie|  4|1962-08-23|3451506020|
|   110|   Cheley|ycheley4@shop-pro.jp|  André|  5|1976-12-26|6175439945|
+------+---------+--------------------+-------+---+----------+----------+


In [24]:
result = spark.sql("""
    SELECT * FROM objekt, personer 
        WHERE objekt.saljare = personer.id 
        LIMIT 50
""")
result.show()

+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+------+-----------+--------------------+----------+---+----------+----------+
|adress|antal_rum|  avslutat|balkong|forsaljningspris|hiss| id|kopare|maklararvode|maklare|manadsavgift|saljare| saljstart|storlek|utgangspris|vaning|adress|  efternamn|               email|   fornamn| id|  personnr| telefonnr|
+------+---------+----------+-------+----------------+----+---+------+------------+-------+------------+-------+----------+-------+-----------+------+------+-----------+--------------------+----------+---+----------+----------+
|    75|        6|2020-07-19|      1|        14561000|   1|  1|     5|      218415|      2|        3090|    107|2020-01-25|    137|   12420000|     5|   153|Osbaldstone|eosbaldstone2y@ml...|Marie-noël|107|1976-05-11|3178324649|
|    72|        5|2020-06-23|      1|         5942500|   1|  2|    27|       89138|     

C. Vilken storlek på lägenheter säljer snabbast i Skåne? De som är under 50kvm, mellan 50-100kvm eller över 100kvm?

In [38]:
result = spark.sql("""
SELECT "Mindre än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek < 50
UNION
SELECT "Mellan 50–100 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek BETWEEN 50 AND 100
UNION
SELECT "Större än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek > 100
ORDER BY dagar LIMIT 1
""") 
result.show()

+-----------------+-----+
|          storlek|dagar|
+-----------------+-----+
|Mellan 50–100 kvm|  116|
+-----------------+-----+
